# 处理时间序列

由于 `Pandas` 最初是为金融模型而创建的，因此它拥有一些功能非常强大的日期、时间、带时间索引数据的处理工具。本节将介绍的日期与时间数据主要包含三类。
- **时间戳**表示某个具体的时间点
    - 例如 2015 年 7 月 4 日上午 7点
- **时间间隔**与**周期**
    - 时间间隔表示开始时间点与结束时间点之间的时间长度
        - 例如 2015 年（指的是 2015 年 1 月 1 日至 2015 年 12 月 31 日这段时间间隔）
    - 周期通常是指一种特殊形式的时间间隔，每个间隔长度相同，彼此之间不会重叠
        - 例如，以 24 小时为周期构成每一天
- **时间增量**（`time delta`）或**持续时间**（`duration`）表示精确的时间长度
    - 例如，某程序运行持续时间 22.56 秒

In [1]:
import numpy as np
import pandas as pd

In [2]:
%%html
<style>
  table {margin-left: 0 !important;}
  img {width:30%; height: 30%;}
</style>

## 1. Python的日期与时间工具

在 `Python` 标准库与第三方库中有许多可以表示日期、时间、时间增量和时间跨度（timespan）的工具。  
尽管 `Pandas` 提供的时间序列工具更适合用来处理数据科学问题，但是了解 `Pandas` 与 `Python` 标准库以及第三方库中的其他时间序列工具之间的关联性将大有裨益。

### 1.1. 原生Python的日期与时间工具：datetime与dateutil

`Python` 基本的日期与时间功能都在标准库的 `datetime` 模块中。  
如果和第三方库 `dateutil` 模块搭配使用，可以快速实现许多处理日期与时间的功能。  
还有一个值得关注的程序包是 `pytz`，这个工具解决了绝大多数时间序列数据都会遇到的难题：时区。

In [3]:
from datetime import datetime
from dateutil import parser

In [4]:
datetime(year=2015, month=7, day=4)  # 用 datetime 类型创建一个日期

datetime.datetime(2015, 7, 4, 0, 0)

In [5]:
date = parser.parse("4th of July, 2015")
date

datetime.datetime(2015, 7, 4, 0, 0)

In [6]:
date.strftime('%A')  # 打印出这一天是星期几

'Saturday'

`datetime` 和 `dateutil` 模块在灵活性与易用性方面都表现出色，你可以用这些对象及其相应的方法轻松完成你感兴趣的任意操作。但如果你处理的时间数据量比较大，那么速度就会比较慢。  
就像之前介绍过的 `Python` 的原生列表对象没有 `NumPy` 中已经被编码的数值类型数组的性能好一样，`Python` 的原生日期对象同样也没有 `NumPy` 中已经被编码的日期（`encoded dates`）类型数组的性能好。

### 1.2. 时间类型数组：NumPy的datetime64类型

`Python` 原生日期格式的性能弱点促使 `NumPy` 团队为 `NumPy` 增加了自己的时间序列类型。  
`datetime64` 类型将日期编码为 64 位整数，这样可以让日期数组非常紧凑（节省内存）。

In [7]:
date = np.array('2015-07-04', dtype=np.datetime64)  # datetime64 需要在设置日期时确定具体的输入类型
date

array('2015-07-04', dtype='datetime64[D]')

In [8]:
date + np.arange(12)

array(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
       '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
       '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
      dtype='datetime64[D]')

因为 `NumPy` 的 `datetime64` 数组内元素的类型是统一的，所以这种数组的运算速度会比 `Python` 的 `datetime` 对象的运算速度快很多，尤其是在处理较大数组时。

`datetime64` 与 `timedelta64` 对象的一个共同特点是，它们都是在**基本时间单位**（`fundamental time unit`）的基础上建立的。  
由于 `datetime64` 对象是 64 位精度，所以可编码的时间范围可以是基本单元的 2<sup>64</sup> 倍。  
也就是说，`datetime64` 在**时间精度**（`time resolution`）与**最大时间跨度**（`maximum time span`）之间达成了一种平衡。

比如你想要一个时间纳秒（`nanosecond，ns`）级的时间精度，那么你就可以将时间编码到 0~2<sup>64</sup> 纳秒或 600 年之内，`NumPy` 会自动判断输入时间需要使用的时间单位。

In [9]:
np.datetime64('2015-07-04')        # 以天为单位的日期

numpy.datetime64('2015-07-04')

In [10]:
np.datetime64('2015-07-04 12:00')  # 以分钟为单位的日期，需要注意的是，时区将自动设置为执行代码的操作系统的当地时区

numpy.datetime64('2015-07-04T12:00')

In [11]:
np.datetime64('2015-07-04 12:59:59.50', 'ns')  # 可以通过各种格式的代码设置基本时间单位。例如，将时间单位设置为纳秒

numpy.datetime64('2015-07-04T12:59:59.500000000')

**日期与时间单位格式代码**

| 代码 | 含义 | 时间跨度 (相对) | 时间跨度 (绝对) |
| -- | -- | -- | -- |
| Y | 年（year） | ± 9.2e18 年 | [9.2e18 BC, 9.2e18 AD] |
| M | 月（month） | ± 7.6e17 年 | [7.6e17 BC, 7.6e17 AD] |
| W | 周（week） | ± 1.7e17 年 | [1.7e17 BC, 1.7e17 AD] |
| D | 日（day） | ± 2.5e16 年 | [2.5e16 BC, 2.5e16 AD] |
| h | 时（hour） | ± 1.0e15 年 | [1.0e15 BC, 1.0e15 AD] |
| m | 分（minute） | ± 1.7e13 年 | [1.7e13 BC, 1.7e13 AD] |
| s | 秒（second） | ± 2.9e12 年 | [ 2.9e9 BC, 2.9e9 AD] |
| ms | 毫秒（millisecond） | ± 2.9e9 年 | [ 2.9e6 BC, 2.9e6 AD] |
| us | 微秒（microsecond） | ± 2.9e6 年 | [290301 BC, 294241 AD] |
| ns | 纳秒（nanosecond） | ± 292 年 | [ 1678 AD, 2262 AD] |
| ps | 皮秒（picosecond） | ± 106 天 | [ 1969 AD, 1970 AD] |
| fs | 飞秒（femtosecond） | ± 2.6 小时 | [ 1969 AD, 1970 AD] |
| as | 原秒（attosecond） | ± 9.2 秒 | [ 1969 AD, 1970 AD] |

对于日常工作中的时间数据类型，默认单位都用纳秒 `datetime64[ns]`，因为用它来表示时间范围精度可以满足绝大部分需求。

最后还需要说明一点，虽然 `datetime64` 弥补了 `Python` 原生的 `datetime` 类型的不足，  
但它缺少了许多 `datetime`（尤其是 `dateutil`）原本具备的便捷方法与函数，  
具体内容请参考 `NumPy` 的 `datetime64` 文档（http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html）。

### 1.3. Pandas的日期与时间工具：理想与现实的最佳解决方案

`Pandas` 所有关于日期与时间的处理方法全部都是通过 `Timestamp` 对象实现的，它利用 `numpy.datetime64` 的有效存储和向量化接口将 `datetime` 和 `dateutil` 的易用性有机结合起来。  
`Pandas` 通过一组 `Timestamp` 对象就可以创建一个可以作为 `Series` 或 `DataFrame` 索引的 `DatetimeIndex`。

In [12]:
date = pd.to_datetime("4th of July, 2015")  # 用 Pandas 的方式演示前面介绍的日期与时间功能
date

Timestamp('2015-07-04 00:00:00')

In [13]:
date.strftime('%A')

'Saturday'

In [14]:
date + pd.to_timedelta(np.arange(12), 'D')

DatetimeIndex(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
               '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
               '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
              dtype='datetime64[ns]', freq=None)

## 2. Pandas时间序列：用时间作索引

`Pandas` 时间序列工具非常适合用来处理带时间戳的索引数据。

In [15]:
index = pd.DatetimeIndex(['2014-07-04', '2014-08-04', '2015-07-04', '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index=index)  # 通过一个时间索引数据创建一个 Series 对象
data

2014-07-04    0
2014-08-04    1
2015-07-04    2
2015-08-04    3
dtype: int64

In [16]:
data['2014-07-04':'2015-07-04']  # 直接用日期进行切片取值

2014-07-04    0
2014-08-04    1
2015-07-04    2
dtype: int64

In [17]:
data['2015']  # 通过年份切片获取该年的数据

2015-07-04    2
2015-08-04    3
dtype: int64

## 3. Pandas时间序列数据结构

本节将介绍 `Pandas` 用来处理时间序列的基础数据类型。
- 针对时间戳数据，`Pandas` 提供了 `Timestamp` 类型。与前面介绍的一样，它本质上是 `Python` 的原生 `datetime` 类型的替代品，但是在性能更好的 `numpy.datetime64` 类型的基础上创建。对应的索引数据结构是 `DatetimeIndex`。
- 针对时间周期数据，`Pandas` 提供了 `Period` 类型。这是利用 `numpy.datetime64` 类型将固定频率的时间间隔进行编码。对应的索引数据结构是 `PeriodIndex`。
- 针对时间增量或持续时间，`Pandas` 提供了 `Timedelta` 类型。`Timedelta` 是一种代替 `Python` 原生 `datetime.timedelta` 类型的高性能数据结构，同样是基于 `numpy.timedelta64` 类型。对应的索引数据结构是 `TimedeltaIndex`。

最基础的日期 / 时间对象是 `Timestamp` 和 `DatetimeIndex`。这两种对象可以直接使用，最常用的方法是 `pd.to_datetime()` 函数，它可以解析许多日期与时间格式。  
对 `pd.to_datetime()` 传递一个日期会返回一个 `Timestamp` 类型，传递一个时间序列会返回一个 `DatetimeIndex` 类型。

In [18]:
dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015', '2015-Jul-6', '07-07-2015', '20150708'])
dates

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-06', '2015-07-07',
               '2015-07-08'],
              dtype='datetime64[ns]', freq=None)

In [19]:
dates.to_period('D')  # 任何 DatetimeIndex 类型都可以通过 to_period() 方法和一个频率代码转换成 PeriodIndex 类型。

PeriodIndex(['2015-07-03', '2015-07-04', '2015-07-06', '2015-07-07',
             '2015-07-08'],
            dtype='period[D]')

In [20]:
dates - dates[0]      # 当用一个日期减去另一个日期时，返回的结果是 TimedeltaIndex 类型

TimedeltaIndex(['0 days', '1 days', '3 days', '4 days', '5 days'], dtype='timedelta64[ns]', freq=None)

### 3.1. 有规律的时间序列：pd.date_range()

## 4. 时间频率与偏移量

**Pandas频率代码**

| 代码 | 描述 | 代码 | 描述 |
| -- | -- | -- | -- |
| D | 天（calendar day，按日历算，含双休日） | B | 天（business day，仅含工作日） |
| W | 周（weekly） |
| M | 月末（month end） | BM | 月末（business month end，仅含工作日） |
| Q | 季末（quarter end） | BQ | 季末（business quarter end，仅含工作日） |
| A | 年末（year end） | BA | 年末（business year end，仅含工作日） |
| H | 小时（hours） | BH | 小时（business hours，工作时间） |
| T | 分钟（minutes） |
| S | 秒（seconds） |
| L | 毫秒（milliseonds） |
| U | 微秒（microseconds） |
| N | 纳秒（nanoseconds） |

**带开始索引的频率代码**

| 代码 | 频率 |
| -- | -- |
| MS | 月初（month start） |
| BMS | 月初（business month start，仅含工作日） |
| QS | 季初（quarter start） |
| BQS | 季初（business quarter start，仅含工作日） |
| AS | 年初（year start） |
| BAS | 年初（business year start，仅含工作日） |

## 5. 重新取样、迁移和窗口

### 5.1. 重新取样与频率转换

### 5.2. 时间迁移

### 5.3. 移动时间窗口

## 6. 案例：美国西雅图自行车统计数据的可视化

### 6.1. 数据可视化

### 6.2. 深入挖掘数据